In [1]:
!nvidia-smi

Wed Aug 24 13:58:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  Off |
| 60%   58C    P8    36W / 450W |    428MiB / 24564MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 165 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 9.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=ce063e50935fb2534ce5cfc3740c4a2650a297dac5be9bac9d19d076bb606aa8
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 24.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [3]:
!pip install git+https://github.com/SKTBrain/KoBERT.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-req-build-li1m4jvt
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-req-build-li1m4jvt
     |████████████████████████████████| 129 kB 32.5 MB/s 
     |████████████████████████████████| 54.7 MB 27 kB/s 
     |████████████████████████████████| 4.5 MB 57.9 MB/s 
     |████████████████████████████████| 1.2 MB 58.0 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x3a064000 @  0x7f3c5f7bf615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 16 kB/s 
     |██████████████████

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook, notebook
import pandas as pd

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda:0")

In [6]:
device

device(type='cuda', index=0)

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/iot4h/Projects/KU-Hackathon-DL-Server/.cache/kobert_v1.zip
using cached model. /home/iot4h/Projects/KU-Hackathon-DL-Server/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
train_data = pd.read_csv('train.csv')

In [9]:
#df=pd.read_csv('/content/drive/MyDrive/server/models/pretrained.csv')
#train_data=pd.read_csv('/content/drive/MyDrive/KU-Hackathon-DL-Server/fci_train_val.txt', sep='\t')

In [10]:
len(train_data)

114289

In [11]:
train_data = train_data.drop(labels='Unnamed: 0', axis=1)

In [12]:
train_data.columns = ['Emotion', 'Sentence']

In [13]:
train_data.head()

,Emotion,Sentence
0,0,아내가 드디어 출산하게 되어서 정말 신이 나.
1,0,아 지금 정말 신이 나.
2,0,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
3,1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
4,1,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.


In [15]:
# from sklearn.model_selection import train_test_split

# train_data, test_data = train_test_split(df, test_size=0.1, shuffle=True, random_state=34)


ModuleNotFoundError: No module named 'sklearn'

In [14]:
test_data = pd.read_csv('test.csv')

In [15]:
# test_data=pd.read_csv('/content/drive/MyDrive/KU-Hackathon-DL-Server/fci_test.txt', sep = '\t')

In [16]:
test_data

,Unnamed: 0,Sentiment,Sentence
0,0,4,요즘 부모님과 많이 부딪혀.
1,1,4,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...
2,2,4,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.
3,4,3,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.
4,5,3,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...
...,...,...,...
14280,20513,4,당장 내일 밥은 어떡하고 내 병원비는 어쩌지.
14281,20514,4,어떻게 해야 할 지 모르겠어. 눈 앞이 캄캄해.
14282,20516,4,자식들은 키워 준 은혜도 모르고 내게 오지도 않네. 너무 외롭고 슬퍼.
14283,20517,4,응 이제 다들 독립해가지고 명절 아니면 찾아오질 않아. 안부전화라도 해주면 좋을 텐데.


In [17]:
test_data = test_data.drop(labels='Unnamed: 0', axis=1)
test_data.columns = ['Emotion', 'Sentence']
test_data.head()

,Emotion,Sentence
0,4,요즘 부모님과 많이 부딪혀.
1,4,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...
2,4,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.
3,3,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.
4,3,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...


In [18]:
print(len(test_data), len(train_data))

14285 114289


In [19]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
            
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    def __len__(self):
        return (len(self.labels))

In [20]:
max_len = 255
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [21]:
tokenizer = get_tokenizer()
tok=nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /home/iot4h/Projects/KU-Hackathon-DL-Server/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [22]:
train_list = []
for q, label in zip(train_data['Sentence'], train_data['Emotion']) :
    data =[]
    data.append(q)
    data.append(str(label))
    if len(q)>1:
      train_list.append(data)
test_list = []
for q, label in zip(test_data['Sentence'], test_data['Emotion']) :
    data =[]
    data.append(q)
    data.append(str(label))
    if len(q)>1:
      test_list.append(data)

In [23]:
data_train = BERTDataset(train_list, 0,1, tok, max_len, True, False)
data_test = BERTDataset(test_list, 0, 1, tok, max_len, True, False)

In [24]:
data_train[0]

(array([   2, 3096, 5330, 1788, 5947, 6855, 4575, 7784, 1767, 6553, 4102,
        3010, 7096, 1370,  517,   54,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [25]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                bert,
                hidden_size = 768,
                num_classes = 7,
                dr_rate = None,
                params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
            
    def gen_attention_mask (self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)



In [27]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [29]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [30]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [31]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
 

In [33]:
train_dataloader

# KoBERT 모델 학습시키기

In [34]:
best_epoch = 0
best_acc = 0.0
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    if test_acc > best_acc:
        best_acc =  test_acc / (batch_id+1)
        best_epoch = e+1
        save_name = 'bestmodel.pth'
        path = f'{save_name}'
        torch.save(model.state_dict(), path)

    print(f'Best epoch : {best_epoch}, best acc : {best_acc}')

  0%|          | 0/3572 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.02679181098938 train acc 0.125
epoch 1 batch id 201 loss 1.917420506477356 train acc 0.16013681592039802
epoch 1 batch id 401 loss 1.834553837776184 train acc 0.1864869077306733
epoch 1 batch id 601 loss 1.7496246099472046 train acc 0.22826539101497503
epoch 1 batch id 801 loss 1.2889468669891357 train acc 0.26283551810237205
epoch 1 batch id 1001 loss 1.4925594329833984 train acc 0.29270729270729273
epoch 1 batch id 1201 loss 1.1030112504959106 train acc 0.3183805162364696
epoch 1 batch id 1401 loss 1.2597867250442505 train acc 0.3398019271948608
epoch 1 batch id 1601 loss 1.5278823375701904 train acc 0.3574133354153654
epoch 1 batch id 1801 loss 1.6648727655410767 train acc 0.37054067184897277
epoch 1 batch id 2001 loss 1.2960455417633057 train acc 0.38205897051474264
epoch 1 batch id 2201 loss 0.9867748022079468 train acc 0.39107223989095863
epoch 1 batch id 2401 loss 1.39999520778656 train acc 0.39881820074968766
epoch 1 batch id 2601 loss 1.19299757480621

  0%|          | 0/447 [00:00<?, ?it/s]

epoch 1 test acc 0.475063457236276
Best epoch : 1, best acc : 0.475063457236276


  0%|          | 0/3572 [00:00<?, ?it/s]

...

epoch 9 test acc 0.5358103166408535
Best epoch : 9, best acc : 0.5358103166408535


  0%|          | 0/3572 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.4731658101081848 train acc 0.90625
epoch 10 batch id 201 loss 0.7248495221138 train acc 0.8622512437810945
epoch 10 batch id 401 loss 0.8479724526405334 train acc 0.8652587281795511
epoch 10 batch id 601 loss 0.6130037903785706 train acc 0.8640287021630616
epoch 10 batch id 801 loss 0.1626182496547699 train acc 0.8628667290886392
epoch 10 batch id 1001 loss 0.8844588994979858 train acc 0.863449050949051
epoch 10 batch id 1201 loss 0.26124754548072815 train acc 0.8655807660283097
epoch 10 batch id 1401 loss 0.4207180440425873 train acc 0.8664123840114204
epoch 10 batch id 1601 loss 0.5901435613632202 train acc 0.8672704559650218
epoch 10 batch id 1801 loss 0.5958462953567505 train acc 0.8659078289838978
epoch 10 batch id 2001 loss 0.20018988847732544 train acc 0.8669259120439781
epoch 10 batch id 2201 loss 0.14971213042736053 train acc 0.8669070876874148
epoch 10 batch id 2401 loss 0.2844461500644684 train acc 0.8679196168263223
epoch 10 batch id 2601 loss 0.3

  0%|          | 0/447 [00:00<?, ?it/s]

epoch 10 test acc 0.5351811220099811
Best epoch : 10, best acc : 0.5351811220099811


In [39]:
save_name = 'checkpoint.pth'
path = f'/content/drive/MyDrive/server/models/{save_name}'
torch.save(model.state_dict(), path)

In [35]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/iot4h/Projects/KU-Hackathon-DL-Server/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [36]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            print(logits)

            # 0 : 기쁨 , 1 : 불안, 2 : 당황, 3 : 슬픔, 4 : 분노, 5 : 상처
            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("불안이")
            elif np.argmax(logits) == 2:
                test_eval.append("딩황이")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("분노가")
            elif np.argmax(logits) == 5:
                test_eval.append("상처가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [40]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

[-0.6881197  -1.280292   -0.4295037  -0.45090747  2.506138    1.8516948
 -8.345776  ]
>> 입력하신 내용에서 분노가 느껴집니다.


[-0.6881197  -1.280292   -0.4295037  -0.45090747  2.506138    1.8516948
 -8.345776  ]
>> 입력하신 내용에서 분노가 느껴집니다.


[-0.44424972 -0.94332457  0.9159704   0.5019629  -0.92462754  2.3521712
 -8.615399  ]
>> 입력하신 내용에서 상처가 느껴집니다.


[-1.7459335 -1.1119463  0.5950695  1.8163944 -1.611555   3.637273
 -9.723924 ]
>> 입력하신 내용에서 상처가 느껴집니다.


[ 0.24289732  5.206374   -0.09255132 -1.2182418  -1.3101612  -1.7211288
 -7.4918137 ]
>> 입력하신 내용에서 불안이 느껴집니다.


[-0.68888015  1.1215366  -0.44157287 -0.94378406 -0.79135835  3.0483458
 -7.7461095 ]
>> 입력하신 내용에서 상처가 느껴집니다.


[ 0.20039998  2.1115503   1.9656788  -1.5480256   0.5367147  -2.1239457
 -8.352742  ]
>> 입력하신 내용에서 불안이 느껴집니다.


[ 0.20039998  2.1115503   1.9656788  -1.5480256   0.5367147  -2.1239457
 -8.352742  ]
>> 입력하신 내용에서 불안이 느껴집니다.




KeyboardInterrupt: Interrupted by user

In [39]:
!python --version

Python 3.9.0
